# PhiAgent

In [2]:
from agent.PhiAgent import PhiAgent
import datetime
from typing import List, Optional
from system_prompt import DEFAULT_SYSTEM_PROMPT
from llama_index.core.tools import FunctionTool
from web_search import (
    get_page_content,
    process_search_results,
)
from agent.PhiAgent import ResponseStreamEvent, ToolStreamEvent
from get_llm import get_llm, get_llm_azure


web_search_tool = FunctionTool.from_defaults(
    fn=process_search_results,
    name="web_search_tool",
    description=(
        f"Used to retrieve information about up-to-date information in {str(datetime.datetime.now().date())}, \
            website or the information out of LLM's knowledge."
    ),
)

web_fetch_tool = FunctionTool.from_defaults(
    fn=get_page_content,
    name="web_fetch_tool",
    description="Used to fetch information of specific url",
)

tools = [web_fetch_tool, web_search_tool]


In [4]:
DEFAULT_SYSTEM_PROMPT_WITH_TIME = DEFAULT_SYSTEM_PROMPT.format(
    date=str(datetime.datetime.now().date())
)
model_name = "gpt-4o-mini"


def run_agent(
    model_name: str,
    system_prompt: str,
    tools: Optional[List] = None,
    history: Optional[List] = None,
):
    if model_name in ["gpt-4o-mini", "gpt-4-turbo", "gpt-4"]:
        agent = PhiAgent(
            tools=tools,
            system_prompt=system_prompt,
            # chat_history=history,
            llm=get_llm_azure(),
            verbose=False,
            timeout=60,
        )
    return agent


# agent.reset()
agent = run_agent(
    model_name=model_name,
    system_prompt=DEFAULT_SYSTEM_PROMPT_WITH_TIME,
    tools=tools,
    history=None,
)


In [7]:
input = "kết quả trận T1 vs BLG mới nhất"
handler = agent.run(input=input, timeout=60)

async for ev in handler.stream_events():
    if isinstance(ev, ToolStreamEvent):
        print("**Reasoning:**")
        print(ev.text)
    elif isinstance(ev, ResponseStreamEvent):
        print(ev.text, end="")


**Reasoning:**
[ToolSelection(tool_id='call_NKCHuruqUjh8R7gTMesPCylF', tool_name='web_search_tool', tool_kwargs={'input': 'T1 vs BLG latest match result November 2024'})]
sub question: ['T1 vs BLG latest match result November 2024']
**Reasoning:**
[ToolSelection(tool_id='call_OlyhZMgQixeI5usgHC3WlAH6', tool_name='web_fetch_tool', tool_kwargs={'input': 'https://www.gameleap.com/articles/worlds-2024-grand-final-t1-vs-blg-match-results'}), ToolSelection(tool_id='call_PYnOZHcM3peTPTOrMaIOo86q', tool_name='web_fetch_tool', tool_kwargs={'input': 'https://www.sportskeeda.com/esports/t1-vs-bilibili-gaming-league-legends-worlds-2024-grand-final-prediction-livestream-details'}), ToolSelection(tool_id='call_2lrp4qhjD5G1WBImxHGUoZYO', tool_name='web_fetch_tool', tool_kwargs={'input': 'https://scores24.live/en/lol/m-12-05-2024-bilibili-gaming-t1'})]
The latest match between T1 and Bilibili Gaming (BLG) occurred on **November 2, 2024**, during the **2024 League of Legends World Championship Grand Fi